Como Atividade M1, resolver novamente o exercício da Atividade 2 (conjunto de dados in e out reais contendo 99 amostras), porém, ao invés de identificar os coeficientes usando otimização linear (através da \\), usar otimização não linear. Dividir a Atividade M1 em 2 etapas:

A. Construir uma function com as seguintes características:

1. argumentos (incógnitas) da function: os valores de todos os coeficientes do polinômio de memória (MP);

2. processamentos internos dentro da function:





carregar os sinais de treinamento (in e out);



processar o sinal de entrada ao longo do MP (observe que dentro dessa function assume-se que os coeficientes são conhecidos, uma vez que esses valores são passados como argumentos da function);



obter a saída estimada pelo MP e calcular o vetor de erro entre saída desejada e saída estimada pela rede;

3. retorno da function: vetor de erro.

B. Usar otimização não linear para encontrar de uma só vez os valores de todos os coeficientes do polinômio de memória (MP). No Matlab, isso é feito pelo comando lsqnonlin, disponível no Optimization toolbox. No Python eu acredito que também existe um equivalente ao lsqnonlin, mas não sei te passar maiores detalhes. Dicas (que se aplicam ao Matlab):





ver um exemplo disponível no help: help lsqnonlin;



no comando lsqnonlin, você chamará a function já pronta na parte A dessa Atividade M1;



você deve inserir uma estimativa inicial para o valor de cada coeficiente do MP. Sugiro você testar por exemplo: a) tudo zero; b) tudo um; c) valores aleatórios;



ao rodar o lsqnonlin, a cada iteração automaticamente são atualizados os valores dos coeficientes. Você pode visualizar, iteração por iteração, como está o MSE (resnorm) a cada iteração usando 'Display' "iter" dentro de optimoptions. (outra dica: fazendo help optimoptions você poderá alterar vários parâmetros do otimizador).

In [4]:
from functions import *
mat = loadmat('IN_OUT_PA.mat')

in_data = mat['in'].reshape(1,-1)[0]
out_data = mat['out'].reshape(1,-1)[0]

In [5]:
import numpy as np

def erro_mp(coef, x_in, y_out, ordem, memoria):
    """
    coef   : vetor de coeficientes do MP
    x_in   : sinal de entrada
    y_out  : sinal de saída desejada
    ordem  : ordem do polinômio (p)
    memoria: número de atrasos (M)
    """
    N = len(x_in)
    y_est = np.zeros(N)

    # Construção do polinômio de memória
    for n in range(memoria, N):
        soma = 0
        idx = 0
        for m in range(memoria + 1):
            for p in range(1, ordem + 1):
                soma += coef[idx] * (x_in[n - m] ** p)
                idx += 1
        y_est[n] = soma

    erro = y_out - y_est
    return erro


In [ ]:
from scipy.optimize import least_squares

ordem = 3       
memoria = 2     
num_coef = ordem * (memoria + 1)

x0 = np.zeros(num_coef)   

res = least_squares(
    erro_mp,
    x0,
    args=(in_data, out_data, ordem, memoria),
    verbose=2  
)
residuos = res.fun  

mse = np.mean(residuos**2)
print("Coeficientes otimizados:", res.x)
print("MSE final:", mse)


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         6.2259e+00                                    1.63e+01    
       1              2         2.3746e-01      5.99e+00       9.55e-01       4.64e-09    
`gtol` termination condition is satisfied.
Function evaluations 2, initial cost 6.2259e+00, final cost 2.3746e-01, first-order optimality 4.64e-09.
Coeficientes otimizados: [ 0.84151459 -0.11441298 -0.39725129  0.07723682  0.11573277  0.07450171
 -0.05220898  0.05259112 -0.04325817]
MSE final: 0.004797235224108902
